## DWNN-RLS

In [1]:
import csv
import math
import random
import h5py as h5py
import pandas as pd
import numpy as np
import scipy.io as scio
import matplotlib.pyplot as plt
import sortscore
from MakeSimilarityMatrix import MakeSimilarityMatrix

In [2]:
with h5py.File('./Data/circRNA_disease_from_circRNADisease/association.h5', 'r') as hf:
        circrna_disease_matrix = hf['infor'][:]

In [3]:
df = pd.DataFrame(circrna_disease_matrix)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       312 non-null    int64
 1   1       312 non-null    int64
 2   2       312 non-null    int64
 3   3       312 non-null    int64
 4   4       312 non-null    int64
 5   5       312 non-null    int64
 6   6       312 non-null    int64
 7   7       312 non-null    int64
 8   8       312 non-null    int64
 9   9       312 non-null    int64
 10  10      312 non-null    int64
 11  11      312 non-null    int64
 12  12      312 non-null    int64
 13  13      312 non-null    int64
 14  14      312 non-null    int64
 15  15      312 non-null    int64
 16  16      312 non-null    int64
 17  17      312 non-null    int64
 18  18      312 non-null    int64
 19  19      312 non-null    int64
 20  20      312 non-null    int64
 21  21      312 non-null    int64
 22  22      312 non-null    int64
 23  23      312 non

In [5]:
df.values.sum()

331

In [6]:
index_tuple = (np.where(circrna_disease_matrix == 1))
one_list = list(zip(index_tuple[0], index_tuple[1]))
random.shuffle(one_list)
split = math.ceil(len(one_list) / 5)

all_tpr = []
all_fpr = []
all_recall = []
all_precision = []
all_accuracy = []
all_F1 = []

# 5-fold start
fold = 1
for i in range(0,len(one_list),split):
    test_index = one_list[i:i+split]
    new_circrna_disease_matrix = circrna_disease_matrix.copy()
    # 抹除已知关系
    for index in test_index:
        new_circrna_disease_matrix[index[0], index[1]] = 0
    roc_circrna_disease_matrix = new_circrna_disease_matrix + circrna_disease_matrix
    rel_matrix = new_circrna_disease_matrix
    # 计算相似高斯相似矩阵
    make_sim_matrix = MakeSimilarityMatrix(rel_matrix)
    circ_gipsim_matrix, dis_gipsim_matrix = make_sim_matrix.circsimmatrix,make_sim_matrix.dissimmatrix

   #giống cả 3 đoạn trên
    dis_sim_matrix = dis_gipsim_matrix
    circ_sim_matrix = circ_gipsim_matrix

    # 根据公式初步写一下
    # 先求两个相似矩阵的特征值和特征向量
    dis_eigenvalues, dis_eigenvectors = np.linalg.eig(dis_sim_matrix)
    circ_eigenvalues, circ_eigenvectors = np.linalg.eig(circ_sim_matrix)
    circ_diag_matrix = np.diag(circ_eigenvalues)
    dis_diag_matrix = np.diag(dis_eigenvalues)

    # 首先计算向量Z
    vectorz_part1 = np.kron(circ_diag_matrix, dis_diag_matrix)
    vectorz_part2 = np.kron(circ_diag_matrix, dis_diag_matrix) + 0.2 * np.eye(vectorz_part1.shape[0])
    #求vectorz_part2的逆运算
    vectorz_part2 = np.asarray(np.asmatrix(vectorz_part2).I)

    vectorz_part3 = np.dot(np.dot(dis_eigenvectors.T, rel_matrix.T), circ_eigenvectors)
    # 把vectorz_part3 的列向量堆叠起来
    for col in range(vectorz_part3.shape[1]):
        if col==0:
            temp_vector = vectorz_part3[:,col]
        else:
            temp_vector = np.concatenate((temp_vector, vectorz_part3[:,col]))
    vectorz_part3 = temp_vector

    vectorz = np.dot(np.dot(vectorz_part1, vectorz_part2), vectorz_part3)
    # # 把vectorz还原为89*533的矩阵
    # matrix_z = np.zeros((89,533))
    # for j in range(matrix_z.shape[1]):
    #     matrix_z[:,j] = vectorz[j*89:j*89+89]

    # matrix_z = np.zeros((62, 514))
    # for j in range(matrix_z.shape[1]):
    #     matrix_z[:, j] = vectorz[j * 62:j * 62 + 62]

    # matrix_z = np.zeros((40, 312))
    # for j in range(matrix_z.shape[1]):
    #     matrix_z[:, j] = vectorz[j * 40:j * 40 + 40]

    matrix_z = np.zeros((rel_matrix.shape[1], rel_matrix.shape[0]))
    for j in range(matrix_z.shape[1]):
        matrix_z[:, j] = vectorz[j * rel_matrix.shape[1]:j * rel_matrix.shape[1] + rel_matrix.shape[1]]

    prediction_matrix = np.dot(np.dot(circ_eigenvectors, matrix_z.T), dis_eigenvectors.T)

    # giống nhau cả 3 methods
    aa = prediction_matrix.shape
    bb = roc_circrna_disease_matrix.shape
    zero_matrix = np.zeros((prediction_matrix.shape[0], prediction_matrix.shape[1]))
    print(prediction_matrix.shape)
    print(roc_circrna_disease_matrix.shape)

    score_matrix_temp = prediction_matrix.copy()
    score_matrix = score_matrix_temp + zero_matrix
    minvalue = np.min(score_matrix)
    score_matrix[np.where(roc_circrna_disease_matrix == 2)] = minvalue - 20
    sorted_circrna_disease_matrix, sorted_score_matrix, sort_index = sortscore.sort_matrix(score_matrix,
                                                                               roc_circrna_disease_matrix)

    tpr_list = []
    fpr_list = []
    recall_list = []
    precision_list = []
    accuracy_list = []
    F1_list = []
    for cutoff in range(sorted_circrna_disease_matrix.shape[0]):
        P_matrix = sorted_circrna_disease_matrix[0:cutoff + 1, :]
        N_matrix = sorted_circrna_disease_matrix[cutoff + 1:sorted_circrna_disease_matrix.shape[0] + 1, :]
        TP = np.sum(P_matrix == 1)
        FP = np.sum(P_matrix == 0)
        TN = np.sum(N_matrix == 0)
        FN = np.sum(N_matrix == 1)
        tpr = TP / (TP + FN)
        fpr = FP / (FP + TN)
        tpr_list.append(tpr)
        fpr_list.append(fpr)
        recall = TP / (TP + FN)
        precision = TP / (TP + FP)
        recall_list.append(recall)
        precision_list.append(precision)
        accuracy = (TN + TP) / (TN + TP + FN + FP)
        F1 = (2 * TP) / (2 * TP + FP + FN)
        F1_list.append(F1)
        accuracy_list.append(accuracy)

    with open('./Data/large_circRNA_cancer/circRNA_list.csv','r') as hs:
        circRNA_list = []
        reader = csv.reader(hs)
        for row in reader:
            circRNA_list.append(row[0].lower())
    with open('./Data/large_circRNA_cancer/cancer_list.csv','r') as rs:
        cancer_list = []
        reader = csv.reader(rs)
        for row in reader:
            cancer_list.append(row[0].lower())
    # 下面是对top50，top100，top200的预测准确的计数
    top_list = [10, 20, 50, 100, 200]
    for num in top_list:
        P_matrix = sorted_circrna_disease_matrix[0:num, :]
        N_matrix = sorted_circrna_disease_matrix[num:sorted_circrna_disease_matrix.shape[0] + 1, :]
        top_count = np.sum(P_matrix == 1)
        index_tuple = np.where(P_matrix == 1)
        tmp_list = list(zip(index_tuple[0], index_tuple[1]))
        for tmp in tmp_list:
            with open(
                    './Data/large_circRNA_cancer/circRNA_cancer_fold{}/DWNN-RLS_dataset1_fold{}_top{}.txt'.format(fold, fold, num),
                    'a') as fs:
                circRNA_index = sort_index[tmp[0], tmp[1]]
                cancer_index = tmp[1]
                fs.write('(' + str(circRNA_index) + ',' + str(cancer_index) + ')' + ' ' + str(
                    circRNA_list[circRNA_index] + ' ' + str(cancer_list[cancer_index] + ' ' + str(
                        prediction_matrix[cancer_index][cancer_index]) + '\n')))
        print("top" + str(num) + ": " + str(top_count))
    fold += 1

    all_tpr.append(tpr_list)
    all_fpr.append(fpr_list)
    all_recall.append(recall_list)
    all_precision.append(precision_list)
    all_accuracy.append(accuracy_list)
    all_F1.append(F1_list)

tpr_arr = np.array(all_tpr)
fpr_arr = np.array(all_fpr)
recall_arr = np.array(all_recall)
precision_arr = np.array(all_precision)
accuracy_arr = np.array(all_accuracy)
F1_arr = np.array(all_F1)

mean_cross_tpr = np.mean(tpr_arr, axis=0)  # axis=0
mean_cross_fpr = np.mean(fpr_arr, axis=0)
mean_cross_recall = np.mean(recall_arr, axis=0)
mean_cross_precision = np.mean(precision_arr, axis=0)
mean_cross_accuracy = np.mean(accuracy_arr, axis=0)
# 计算此次五折的平均评价指标数值
mean_accuracy = np.mean(np.mean(accuracy_arr, axis=1), axis=0)
mean_accuracy1 = np.mean(accuracy_arr)
mean_recall = np.mean(np.mean(recall_arr, axis=1), axis=0)
mean_precision = np.mean(np.mean(precision_arr, axis=1), axis=0)
mean_F1 = np.mean(np.mean(F1_arr, axis=1), axis=0)
print("accuracy:%.4f,recall:%.4f,precision:%.4f,F1:%.4f" % (mean_accuracy, mean_recall, mean_precision, mean_F1))

roc_auc = np.trapz(mean_cross_tpr, mean_cross_fpr)
AUPR = np.trapz(mean_cross_precision, mean_cross_recall)
print("AUC:%.4f,AUPR:%.4f" % (roc_auc, AUPR))

# # 存储tpr，fpr,recall,precision
# with h5py.File('./PlotFigure/DWNN-RLS_noncancer_5fold_AUC.h5','w') as hf:
#     hf['fpr'] = mean_cross_fpr
#     hf['tpr'] = mean_cross_tpr
# with h5py.File('./PlotFigure/DWNN-RLS_noncancer_5fold_AUPR.h5','w') as h:
#     h['recall'] = mean_cross_recall
#     h['precision'] = mean_cross_precision

plt.plot(mean_cross_fpr, mean_cross_tpr, label='mean ROC=%0.4f' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc=0)
# plt.savefig("./FinalResultPng/roc-DWNN-RLS_circad.png")
print("runtime over, now is :")
# print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
plt.show()


KeyboardInterrupt: 

In [7]:
index_tuple = (np.where(circrna_disease_matrix == 1))
one_list = list(zip(index_tuple[0], index_tuple[1]))
random.shuffle(one_list)
split = math.ceil(len(one_list) / 5)

all_tpr = []
all_fpr = []
all_recall = []
all_precision = []
all_accuracy = []
all_F1 = []

In [8]:
fold = 1
for i in range(0,len(one_list),split):
    test_index = one_list[i:i+split]
    new_circrna_disease_matrix = circrna_disease_matrix.copy()

    for index in test_index:
        new_circrna_disease_matrix[index[0], index[1]] = 0
    roc_circrna_disease_matrix = new_circrna_disease_matrix + circrna_disease_matrix
    rel_matrix = new_circrna_disease_matrix

    make_sim_matrix = MakeSimilarityMatrix(rel_matrix)
    circ_gipsim_matrix, dis_gipsim_matrix = make_sim_matrix.circsimmatrix,make_sim_matrix.dissimmatrix
    dis_sim_matrix = dis_gipsim_matrix
    circ_sim_matrix = circ_gipsim_matrix

    dis_eigenvalues, dis_eigenvectors = np.linalg.eig(dis_sim_matrix)
    circ_eigenvalues, circ_eigenvectors = np.linalg.eig(circ_sim_matrix)
    circ_diag_matrix = np.diag(circ_eigenvalues)
    dis_diag_matrix = np.diag(dis_eigenvalues)

    vectorz_part1 = np.kron(circ_diag_matrix, dis_diag_matrix)
    vectorz_part2 = np.kron(circ_diag_matrix, dis_diag_matrix) + 0.2 * np.eye(vectorz_part1.shape[0])
    vectorz_part2 = np.asarray(np.asmatrix(vectorz_part2).I)
    vectorz_part3 = np.dot(np.dot(dis_eigenvectors.T, rel_matrix.T), circ_eigenvectors)

    for col in range(vectorz_part3.shape[1]):
        if col==0:
            temp_vector = vectorz_part3[:,col]
        else:
            temp_vector = np.concatenate((temp_vector, vectorz_part3[:,col]))
    vectorz_part3 = temp_vector
    vectorz = np.dot(np.dot(vectorz_part1, vectorz_part2), vectorz_part3)

    matrix_z = np.zeros((rel_matrix.shape[1], rel_matrix.shape[0]))
    for j in range(matrix_z.shape[1]):
        matrix_z[:, j] = vectorz[j * rel_matrix.shape[1]:j * rel_matrix.shape[1] + rel_matrix.shape[1]]

    prediction_matrix = np.dot(np.dot(circ_eigenvectors, matrix_z.T), dis_eigenvectors.T)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3756\1016714710.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix_z[:, j] = vectorz[j * rel_matrix.shape[1]:j * rel_matrix.shape[1] + rel_matrix.shape[1]]


In [13]:
prediction_matrix_real = prediction_matrix.real

In [14]:
result = pd.DataFrame(prediction_matrix_real)

In [15]:
result

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.996465,0.000026,0.000024,0.000002,0.000004,0.000001,0.000001,0.000014,0.000014,0.000008,...,0.000004,0.000004,0.000060,0.000005,0.000001,0.000001,0.000003,0.000006,0.000008,0.000014
1,0.000063,0.991608,0.000059,0.000005,0.000008,0.000003,0.000003,0.000030,0.000030,0.000017,...,0.000008,0.000008,0.000306,0.000011,0.000003,0.000003,0.000007,0.000012,0.000017,0.000030
2,0.996465,0.000026,0.000024,0.000002,0.000004,0.000001,0.000001,0.000014,0.000014,0.000008,...,0.000004,0.000004,0.000060,0.000005,0.000001,0.000001,0.000003,0.000006,0.000008,0.000014
3,0.996465,0.000026,0.000024,0.000002,0.000004,0.000001,0.000001,0.000014,0.000014,0.000008,...,0.000004,0.000004,0.000060,0.000005,0.000001,0.000001,0.000003,0.000006,0.000008,0.000014
4,0.996465,0.000026,0.000024,0.000002,0.000004,0.000001,0.000001,0.000014,0.000014,0.000008,...,0.000004,0.000004,0.000060,0.000005,0.000001,0.000001,0.000003,0.000006,0.000008,0.000014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,0.000007,0.000007,0.000009,0.000006,0.000014,0.000006,0.000006,0.000066,0.000066,0.000030,...,0.000006,0.000006,-0.000008,0.000019,0.983537,0.000005,0.000011,0.000020,0.000030,0.000066
308,0.996465,0.000026,0.000024,0.000002,0.000004,0.000001,0.000001,0.000014,0.000014,0.000008,...,0.000004,0.000004,0.000060,0.000005,0.000001,0.000001,0.000003,0.000006,0.000008,0.000014
309,0.996465,0.000026,0.000024,0.000002,0.000004,0.000001,0.000001,0.000014,0.000014,0.000008,...,0.000004,0.000004,0.000060,0.000005,0.000001,0.000001,0.000003,0.000006,0.000008,0.000014
310,0.000007,0.000007,0.000016,0.000024,0.000044,0.000009,0.000006,0.000159,0.000159,0.000086,...,0.000067,0.000067,0.000087,0.000058,0.000013,0.000009,0.000035,0.000076,0.000086,0.000159


In [26]:
np.savetxt("DWNN-RLS_result.csv", result, delimiter=",")

In [50]:
aa = prediction_matrix.shape
bb = roc_circrna_disease_matrix.shape
zero_matrix = np.zeros((prediction_matrix.shape[0], prediction_matrix.shape[1]))
print(prediction_matrix.shape)
print(roc_circrna_disease_matrix.shape)

score_matrix_temp = prediction_matrix.copy()
score_matrix = score_matrix_temp + zero_matrix
minvalue = np.min(score_matrix)
score_matrix[np.where(roc_circrna_disease_matrix == 2)] = minvalue - 20

(312, 40)
(312, 40)


In [55]:
minvalue.real

-0.35533340658806895

In [110]:
P_matrix = np.where(prediction_matrix_real >= 0.5, 1, 0)  
TP = np.sum(np.logical_and(P_matrix == 1, circrna_disease_matrix == 1))
FP = np.sum(np.logical_and(P_matrix == 1, circrna_disease_matrix == 0))
TN = np.sum(np.logical_and(P_matrix == 0, circrna_disease_matrix == 0))
FN = np.sum(np.logical_and(P_matrix == 0, circrna_disease_matrix == 1))

# Tính True Positive Rate (TPR) và False Positive Rate (FPR)
tpr = TP / (TP + FN)
fpr = FP / (FP + TN)

print(TP) 
print(FP)  
print(TN)  
print(FN)  
print("True Positive Rate (TPR):", tpr)
print("False Positive Rate (FPR):", fpr)


265
0
12149
66
True Positive Rate (TPR): 0.8006042296072508
False Positive Rate (FPR): 0.0


In [108]:
P_matrix_df = pd.DataFrame(P_matrix)
sum_of_values = P_matrix_df.sum().sum()
sum_of_values

306

In [109]:
np.savetxt("DWNN-RLS_test.csv", P_matrix, delimiter=",")